## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_groq import ChatGroq
model=ChatGroq(model="openai/gpt-oss-120b",groq_api_key=groq_api_key)
model

c:\Users\Shubham Prakash\11Drive\myCodes\AI\AgenticAI_KrishNaik\M06_LangchainChatbot\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000014DEC149A90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014DEC14A660>, model_name='openai/gpt-oss-120b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Shubham and I am a Chief AI Engineer")])

AIMessage(content='Hello Shubham! 👋 Great to meet a Chief AI Engineer. How can I assist you today? Whether you’d like to discuss a technical challenge, explore the latest AI research, brainstorm product ideas, or anything else, I’m here to help.', additional_kwargs={'reasoning_content': "We need to respond. The user introduced themselves. Probably we should greet them, ask how can help. Since they're a Chief AI Engineer, maybe discuss AI topics. Follow guidelines: be friendly, ask how can assist."}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 86, 'total_tokens': 192, 'completion_time': 0.21049726, 'prompt_time': 0.002980569, 'queue_time': 0.051540491, 'total_time': 0.213477829}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ed9190d8b7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--0e7aa2bd-9f1b-4782-bcf7-0251c2339b82-0', usage_metadata={'input_tokens': 86, 'output_tokens': 106, 'total_tokens': 192})

In [4]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is Shubham and I am a Chief AI Engineer"),
        AIMessage(content="Hello Shubham! 👋 It’s great to meet you. As a Chief AI Engineer, you must be working on some fascinating projects. How can I assist you today?"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content='Your name is **Shubham**, and you’re a **Chief AI Engineer**. How can I help you today?', additional_kwargs={'reasoning_content': 'The user asks: "Hey What\'s my name and what do I do?" The user earlier said: "Hi , My name is Shubham and I am a Chief AI Engineer". So we respond with that. Also possibly ask if they\'d like anything else. Should be friendly.'}, response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 142, 'total_tokens': 233, 'completion_time': 0.180643297, 'prompt_time': 0.005725, 'queue_time': 0.04747919, 'total_time': 0.186368297}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ed9190d8b7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--205ea222-8908-4a7d-85f3-a3e3a9f91538-0', usage_metadata={'input_tokens': 142, 'output_tokens': 91, 'total_tokens': 233})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Shubham and I am a Chief AI Engineer")],
    config=config
)

In [8]:
response.content

'Hello Shubham! 👋 It’s great to meet a Chief AI Engineer. How can I assist you today? Whether you have technical questions, need brainstorming ideas, or just want to chat about the latest AI trends, I’m here to help.'

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is **Shubham**.', additional_kwargs={'reasoning_content': 'The user asks "What\'s my name?" The user previously said "Hi, My name is Shubham and I am a Chief AI Engineer". So answer: Shubham.'}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 151, 'total_tokens': 206, 'completion_time': 0.109279765, 'prompt_time': 0.005743172, 'queue_time': 0.050895898, 'total_time': 0.115022937}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_213abb2467', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--045dc8bf-4ef0-4a3d-bdfa-adb692211581-0', usage_metadata={'input_tokens': 151, 'output_tokens': 55, 'total_tokens': 206})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'I’m sorry—I don’t have any information about your name. If you’d like, you can tell me what you’d like to be called, and I’ll use that going forward!'

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

'Nice to meet you, John! How can I help you today?'

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'You told me your name is **John**.'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer all the questions to the best of your ability."),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Shubham")]})

AIMessage(content='Hello Shubham! 👋 Nice to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'We need to respond as ChatGPT. The user says "Hi My name is Shubham". We can greet back. Possibly ask how can help. Use friendly tone.'}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 98, 'total_tokens': 162, 'completion_time': 0.130236606, 'prompt_time': 0.003449528, 'queue_time': 0.050902902, 'total_time': 0.133686134}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_3a688838c3', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--fc817750-019b-46a7-8e03-0a54d1c0c0c5-0', usage_metadata={'input_tokens': 98, 'output_tokens': 64, 'total_tokens': 162})

In [15]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Shubham")],
    config=config
)

response

AIMessage(content='Hello Shubham! Nice to meet you. How can I assist you today?', additional_kwargs={'reasoning_content': 'The user says "Hi My name is Shubham". Probably a greeting and introduction. We should respond politely, maybe ask how can I help. Use friendly tone.'}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 98, 'total_tokens': 159, 'completion_time': 0.12280823, 'prompt_time': 0.003637098, 'queue_time': 0.051584141, 'total_time': 0.126445328}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ed9190d8b7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--72042cb9-ad19-496e-b615-1f857cfb1c7d-0', usage_metadata={'input_tokens': 98, 'output_tokens': 61, 'total_tokens': 159})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Shubham.'

In [18]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [19]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Shubham")],"language":"Hindi"})
response.content

'नमस्ते शु्भम! आपसे मिलकर खुशी हुई। आज मैं आपकी किस प्रकार मदद कर सकता हूँ?'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [21]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Shubham")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते शुभम! आपसे मिलकर ख़ुशी हुई। आप कैसे हैं?'

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [23]:
response.content

'आपका नाम **शुभम** है।'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [28]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Shubham"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

ImportError: Could not import transformers python package. This is needed in order to calculate get_token_ids. Please install it with `pip install transformers`.

In [40]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream? 😊🍦\n"

In [41]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "whats 2 + 2" 😊  \n\n\n\n'

In [42]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [43]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I don't have access to past conversations or any personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to know! 😊  \n\n"

In [44]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  Just let me know what it is. \n\n"